In [88]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import pandas as pd
import xmltodict
import json
import datetime
import sqlite3
import os

DB_PATH = os.path.dirname(__file__) + '/CoronaBotDB'

today =datetime.datetime.now()

def five_days_ago(today):
    five = (today-datetime.timedelta(5)).strftime("%Y%m%d")
    return five 

key = 'RI5ekmQZaQtJcWF%2BFp%2FjIPg3kaXeWQj0MfyFVPynolhE9rUNQjg%2FCdWF1GkZe0UWS63SVaRd26nbQxZMqWGfKQ%3D%3D'
url = f'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey={key}&'
queryParams = urlencode({ quote_plus('pageNo') : 1, quote_plus('numOfRows') : 10,quote_plus('startCreateDt') : five_days_ago(today),
                        quote_plus('endCreateDt') : today.strftime("%Y%m%d")})

url2 = url + queryParams
response = urlopen(url2) 
results = response.read().decode("utf-8")

results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))

corona=data['response']['body']['items']['item']

Date=[]
Cnt=[]
clear_cnt=[]
care_cnt=[]
death_cnt=[]
for i in corona:
    Date.append(i['stateDt'])
    Cnt.append(i['decideCnt'])
    clear_cnt.append(i['clearCnt'])
    care_cnt.append(i['careCnt'])
    death_cnt.append(i['deathCnt'])

df=pd.DataFrame([Date,Cnt,clear_cnt,care_cnt,death_cnt]).T
df.columns=['Date','acc_cnt','clear_cnt','care_cnt','death_cnt']
df=df.sort_values(by='Date', ascending=True)

#날짜 별 확진자/사망자 수 구하기
df['acc_cnt']=df['acc_cnt'].astype(int)
df['death_cnt']=df['death_cnt'].astype(int)
df['확진자 수']=(df.acc_cnt-df.acc_cnt.shift()).fillna(0)
df['사망자 수']=(df.death_cnt-df.death_cnt.shift()).fillna(0)

NameError: name '__file__' is not defined

In [84]:
df

,Date,acc_cnt,clear_cnt,care_cnt,death_cnt,확진자 수,사망자 수
5,20201210,40098,30637,8897,564,0.0,0.0
4,20201211,40786,31157,9057,572,688.0,8.0
3,20201212,41736,31493,9665,578,950.0,6.0
2,20201213,42766,31814,10372,580,1030.0,2.0
1,20201214,43484,32102,10795,587,718.0,7.0
0,20201215,44364,32559,11205,600,880.0,13.0


In [87]:
def create_db():
    con = sqlite3.connect(DB_PATH + '/newkorea.db')
    cursor=con.cursor()
    cursor.execute("CREATE TABLE korea(Date text, acc int, clear_a int, care_a int, death_a int, con int, death int)")
    con.commit()
    con.close()
    
def input_db():
    con = sqlite3.connect(DB_PATH + '/newkorea.db')
    cursor=con.cursor()
    length1 = df.shape[0]
    length2 = df.shape[1]
    for i in range(length1):
        row = []
        for j in range(length2):
            row.append(df.iloc[i,j])
        cursor.execute("INSERT INTO korea VALUES(?, ?, ?, ?, ?, ?, ?)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6]))
    con.commit()
    con.close()

NameError: name '__file__' is not defined

In [86]:
create_db()
input_db()

NameError: name 'DB_PATH' is not defined